In [3]:
#Import statements
import pandas as pd
from pandas import datetime as dt
import csv
import numpy as np
#import lib.google_sheet_api as gs
import os

/var/folders/0_/q6d4fymj0v9bl3m8gdcnl6n40000gn/T/ipykernel_27604/1883485394.py:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime as dt


In [7]:
#Inputs:
filename = "/Users/suneetakartha/Downloads/Med Consumption (1).csv"

In [8]:
#Get data
site_interval_data = pd.read_csv(filename)
site_interval_data.columns = ['date', 'time', 'hour', 'month', 'gross_load', 'solar', 'battery','load_net_solar']

site_interval_data['date']=pd.to_datetime(site_interval_data['date'], utc=True)
site_interval_data['solar']=site_interval_data['solar'].fillna(0)
site_interval_data=site_interval_data.groupby(['date','month','hour'])['gross_load','solar','battery'].sum().reset_index()
# data['unit']=unit
# replacement = data.copy()
# replacement['load']
site_interval_data['week'] = site_interval_data['date'] - pd.offsets.Week(weekday=6)
site_interval_data['dow']= site_interval_data['date'].dt.dayofweek
site_interval_data['month']= site_interval_data['date'].dt.month
# site_interval_data['SOC_net']= site_interval_data.groupby(['date'])['solar'].cumsum()

/var/folders/0_/q6d4fymj0v9bl3m8gdcnl6n40000gn/T/ipykernel_27604/2136430867.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  site_interval_data=site_interval_data.groupby(['date','month','hour'])['gross_load','solar','battery'].sum().reset_index()


In [9]:
site_interval_data

,date,month,hour,gross_load,solar,battery,week,dow
0,2017-01-01 00:00:00+00:00,1,0,3.950681,0.0,0.000000,2016-12-25 00:00:00+00:00,6
1,2017-01-01 00:00:00+00:00,1,1,3.217718,0.0,0.000000,2016-12-25 00:00:00+00:00,6
2,2017-01-01 00:00:00+00:00,1,2,2.951792,0.0,0.000000,2016-12-25 00:00:00+00:00,6
3,2017-01-01 00:00:00+00:00,1,3,2.888128,0.0,0.000000,2016-12-25 00:00:00+00:00,6
4,2017-01-01 00:00:00+00:00,1,4,2.910504,0.0,0.000000,2016-12-25 00:00:00+00:00,6
...,...,...,...,...,...,...,...,...
8755,2017-12-31 00:00:00+00:00,12,19,9.576349,0.0,9.097531,2017-12-24 00:00:00+00:00,6
8756,2017-12-31 00:00:00+00:00,12,20,9.041044,0.0,5.478577,2017-12-24 00:00:00+00:00,6
8757,2017-12-31 00:00:00+00:00,12,21,8.277436,0.0,0.000000,2017-12-24 00:00:00+00:00,6
8758,2017-12-31 00:00:00+00:00,12,22,6.895613,0.0,0.000000,2017-12-24 00:00:00+00:00,6


In [11]:
def get_weekly_baseline(x, day_of_bl, hour_start, hour_end):
#     x['dow']= x['Date'].dt.dayofweek
#     weekday_series= x['dow']
 
    moo_hours= x[(x['hour']>= hour_start) & (x['hour']<hour_end)]
    
# The below code takes: 1) Rolling 45 days 2) Find and average the five highest days IF that day is the dow specified in the function 
    
    
    moo_hours['baseline_setting_base']=np.where(moo_hours['dow']==0, moo_hours['gross_load'], 0)
    moo_hours['best_hour_kw'] = moo_hours.groupby(['hour'])['baseline_setting_base'].transform(lambda x:x.rolling(45).apply(lambda x: np.mean(sorted(x,reverse=True)[:5])))
    
    z=moo_hours[moo_hours['dow']<=1]
    z=z[['week','hour','best_hour_kw']]
    moo_hours = moo_hours.merge(right = z, on = ['week', 'hour'])
    
    moo_hours=moo_hours.drop(columns=['best_hour_kw_x'])
    moo_hours=moo_hours.sort_values(['date','hour'])
    moo_hours=moo_hours.drop_duplicates()
    return moo_hours


weekly_bl = get_weekly_baseline(site_interval_data, 0, 17, 19)
                        
# def get_cost_revenue(utility, TOU_time, season):
#     x['month_num']= x['date'].dt.month
#     season = np.where((x['month_num']>=6 and x['month_num']>=9), 'Summer', 'Winter')
    
    


/var/folders/0_/q6d4fymj0v9bl3m8gdcnl6n40000gn/T/ipykernel_27604/564548094.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moo_hours['baseline_setting_base']=np.where(moo_hours['dow']==0, moo_hours['gross_load'], 0)
/var/folders/0_/q6d4fymj0v9bl3m8gdcnl6n40000gn/T/ipykernel_27604/564548094.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moo_hours['best_hour_kw'] = moo_hours.groupby(['hour'])['baseline_setting_base'].transform(lambda x:x.rolling(45).apply(lambda x: np.mean(sorted(x,reverse=True)[:5

In [12]:
weekly_bl

,date,month,hour,gross_load,solar,battery,week,dow,baseline_setting_base,best_hour_kw_y
0,2017-01-02 00:00:00+00:00,1,17,7.939849,0.258555,7.297229,2017-01-01 00:00:00+00:00,0,7.939849,NaN
14,2017-01-02 00:00:00+00:00,1,18,9.596617,0.000000,9.116786,2017-01-01 00:00:00+00:00,0,9.596617,NaN
2,2017-01-03 00:00:00+00:00,1,17,7.677126,0.252981,7.052938,2017-01-01 00:00:00+00:00,1,0.000000,NaN
16,2017-01-03 00:00:00+00:00,1,18,9.371058,0.000000,8.902505,2017-01-01 00:00:00+00:00,1,0.000000,NaN
4,2017-01-04 00:00:00+00:00,1,17,7.689807,0.281195,7.038181,2017-01-01 00:00:00+00:00,2,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...
1450,2017-12-29 00:00:00+00:00,12,18,9.725764,0.000000,9.239476,2017-12-24 00:00:00+00:00,4,0.000000,9.410733
1438,2017-12-30 00:00:00+00:00,12,17,8.447821,0.218098,7.818237,2017-12-24 00:00:00+00:00,5,0.000000,7.975022
1452,2017-12-30 00:00:00+00:00,12,18,9.886771,0.000000,9.392433,2017-12-24 00:00:00+00:00,5,0.000000,9.410733
1440,2017-12-31 00:00:00+00:00,12,17,8.474284,0.232588,7.829611,2017-12-24 00:00:00+00:00,6,0.000000,7.975022


In [14]:
def get_revenue_values(test):
    test['month']= test['date'].dt.month
    test['day']= test['date'].dt.day
    test['year']= test['date'].dt.year
    min_month= (test.groupby(['month','year']).agg({'day':'min',
                                             'hour':'min',
                             'baseline_setting_base': lambda x: (x > 0).mean()})).reset_index()

    data = test.merge(min_month, on = ['year','month','day', 'hour'], how='left')
    data= data.fillna(0)

    data

    _2022_cap_rate=({'month': [1,2,3,4,5,6,7,8,9,10,11,12], 
                     'value':[2970, 2963, 2999, 3037, 5036, 5368, 5472, 5474, 5479, 5298, 2778, 2684]})

    idktest= pd.DataFrame.from_dict(_2022_cap_rate)
    idktest
    # idktest = idktest.rename(columns=('month', 'value'))
    data2 = data.merge(idktest, on = ['month'], how='left')

    data2['cap_rev']= data2['baseline_setting_base_y']/10* data2['value']
    data2

    energy_rev_data=pd.read_csv('/Users/suneetakartha/Downloads/energy_revenue_sizing_data.csv')
    energy_rev_data =pd.melt(energy_rev_data, id_vars=['Month','Day'], value_vars=['17', '18','19','20'], ignore_index=False)
    energy_rev_data.columns=['month', 'day', 'hour', 'energy_rev_val']
    energy_rev_data['hour']=energy_rev_data['hour'].astype('int64')
    final_rev= data2.merge(energy_rev_data, on = ['month','day','hour'], how='left')
    final_rev['energy_rev']=np.where(final_rev['baseline_setting_base_x']>0, 0, final_rev['best_hour_kw_y']/1000* final_rev['energy_rev_val'])
    final_rev=final_rev.drop(columns=['baseline_setting_base_y','value', 'energy_rev_val'])
    return final_rev

In [15]:
test=get_revenue_values(weekly_bl)

In [16]:
test

,date,month,hour,gross_load,solar,battery,week,dow,baseline_setting_base_x,best_hour_kw_y,day,year,cap_rev,energy_rev
0,2017-01-02 00:00:00+00:00,1,17,7.939849,0.258555,7.297229,2017-01-01 00:00:00+00:00,0,7.939849,0.000000,2,2017,0.495,0.000000
1,2017-01-02 00:00:00+00:00,1,18,9.596617,0.000000,9.116786,2017-01-01 00:00:00+00:00,0,9.596617,0.000000,2,2017,0.000,0.000000
2,2017-01-03 00:00:00+00:00,1,17,7.677126,0.252981,7.052938,2017-01-01 00:00:00+00:00,1,0.000000,0.000000,3,2017,0.000,0.000000
3,2017-01-03 00:00:00+00:00,1,18,9.371058,0.000000,8.902505,2017-01-01 00:00:00+00:00,1,0.000000,0.000000,3,2017,0.000,0.000000
4,2017-01-04 00:00:00+00:00,1,17,7.689807,0.281195,7.038181,2017-01-01 00:00:00+00:00,2,0.000000,0.000000,4,2017,0.000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,2017-12-29 00:00:00+00:00,12,18,9.725764,0.000000,9.239476,2017-12-24 00:00:00+00:00,4,0.000000,9.410733,29,2017,0.000,0.272460
738,2017-12-30 00:00:00+00:00,12,17,8.447821,0.218098,7.818237,2017-12-24 00:00:00+00:00,5,0.000000,7.975022,30,2017,0.000,0.296683
739,2017-12-30 00:00:00+00:00,12,18,9.886771,0.000000,9.392433,2017-12-24 00:00:00+00:00,5,0.000000,9.410733,30,2017,0.000,0.298551
740,2017-12-31 00:00:00+00:00,12,17,8.474284,0.232588,7.829611,2017-12-24 00:00:00+00:00,6,0.000000,7.975022,31,2017,0.000,0.269503


In [253]:
# Set baseline of some day within a week, for the month, the value will just be the load average of the month prior
# 

def get_monthly_capacity_quantity(x, day_of_bl, hour_start, hour_end, duration): 
    ## Limit data set to the hours in question
    x= x[(x['hour']>= hour_start) & (x['hour']<hour_end)]
    x['month']=x['date'].to_numpy().astype('datetime64[M]')
    
    ## filter down to the day 
    x=x[x['dow']== day_of_bl]
    
    monthly_demonstrated_capacity = x.groupby(['month'])['gross_load'].mean().shift(duration)
    monthly_demonstrated_capacity = pd.DataFrame(monthly_demonstrated_capacity)
#     baseline_setting_day['best_hour_kw'] = baseline_setting_day.groupby(['month','hour'])['load'].mean()
            
#     x = x.merge(right = baseline_setting_day, on = ['month', 'hour'])
    
    return monthly_demonstrated_capacity  

In [263]:
monthly_bl= get_monthly_capacity_quantity(replacement, 0, 19, 21,1)
monthly_bl


<ipython-input-253-4fb8a8409d4d>:7: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  x['month']=x['date'].to_numpy().astype('datetime64[M]')
<ipython-input-253-4fb8a8409d4d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['month']=x['date'].to_numpy().astype('datetime64[M]')


,gross_load
month,
2020-11-01,NaN
2020-12-01,0.766555
2021-01-01,0.893653
2021-02-01,0.866686
2021-03-01,0.786803
2021-04-01,0.726265
2021-05-01,1.184294
2021-06-01,1.112973
2021-07-01,1.605884


In [264]:
def get_energy_revenues(test, dispatches):
    dispatches = 12
    test = test[['month','two_hour_gen']]
    test['month']=test['month'].astype(int)
    test['dispatches']=dispatches
    test['location']='SLAP_PGCC'
    test['MW']=test['two_hour_gen']/1000
    df = pd.DataFrame()
    for i in range(0, test.shape[0]):
        lmp_data = pd.read_csv('/Users/suneetakartha/Documents/dev/GitHub/slap_lmps/'+"SLAP_PGCC"+'-APND-'+"DAM"+'.csv')
        location = test.iloc[i, 3]
        MW=test.iloc[i,4]
        dispatches=int(test.iloc[i,2])
        month= int(test.iloc[i, 0])

        lmp_data['PRC'] = lmp_data['MW']
        lmp_data= lmp_data[lmp_data>='2020-01-01']
        lmp_data.drop('MW', axis=1)

        lmp_data['start_datetime'] = pd.to_datetime(lmp_data['INTERVALSTARTTIME_GMT'],
                                                        format='%Y-%m-%dT%H:%M:%S') - pd.Timedelta(hours=7)
        lmp_data['month']=lmp_data['start_datetime'].dt.month

        lmp_data = lmp_data[lmp_data['month'] == month]

        bid_price = lmp_data.sort_values('PRC', ascending=False).iloc[(dispatches)-1,17]
        lmp_data['dispatch'] = lmp_data.PRC >= bid_price

        lmp_data['dispatch'] = lmp_data['dispatch'].astype(int)
        lmp_data['dispatch_rev']=0
        lmp_data.loc[
            lmp_data.dispatch == True,
            'dispatch_rev'
        ] = lmp_data.loc[
            lmp_data.dispatch == True,
            'PRC'
        ]
        dispatch_count = sum(lmp_data['dispatch'])
        revenue_mwh = round(sum(lmp_data['dispatch_rev']), 2)
        revenue=round(sum(lmp_data['dispatch_rev'], 2))*MW

        #if dispatches==2:
        #    print(month)
        #    print(lmp_data[lmp_data['dispatch_rev']>0]['OPR_HR'])
        #    print(lmp_data[lmp_data['dispatch_rev']>0]['dispatch_rev'])

        df = df.append(pd.DataFrame([[location, 
                                 unit, 
                                 MW,
                                 dispatches,
                                 month,
                                 bid_price, dispatch_count, revenue_mwh, revenue]],
                                   columns = ['location',
                                             'unit',
                                             'mw',
                                              'dispatch_case',
                                             'month',
                                             'bid_price',
                                             'dispatches',
                                             'revenue_mwh',
                                             'revenue']))
    return df 

In [90]:
total_bl.to_csv('~/Documents/Hacky_PearlX.csv')

In [1]:
SCE_TOU_Cost= []

IndentationError: expected an indented block (<ipython-input-1-156a0ff2b816>, line 26)

In [123]:
replacement_gen

,,two_hour_baseline,7to9_gen
date,dow,,
2020-11-01 00:00:00+00:00,6.0,NaN,1.308712
2020-11-02 00:00:00+00:00,0.0,NaN,1.613767
2020-11-03 00:00:00+00:00,1.0,NaN,1.551822
2020-11-04 00:00:00+00:00,2.0,NaN,2.730427
2020-11-05 00:00:00+00:00,3.0,NaN,1.188736
...,...,...,...
2021-10-27 00:00:00+00:00,2.0,2.031569,1.575751
2021-10-28 00:00:00+00:00,3.0,2.042723,1.371266
2021-10-29 00:00:00+00:00,4.0,2.472451,1.605357


In [125]:
replacement_gen.to_csv('~/Downloads/PearlX_dow_9_act.csv')

In [25]:
#Caclulate Average curtailable load performance for inc + repl
def get_2hour_gen(x, mytype, battery_kw, battery_kwh):
    x['load_limit'] = x['load']
    x['baseline_limit'] = x['baseline_unadj']
    if mytype=='replacement':
        x['fake_soc']=1
    elif mytype == 'incremental':
        x['fake_soc']=x['soc']
    x['battery_limit_kw'] = x['soc']*battery_kw
    x['gen']= pd.concat([x['load_limit'], x['baseline_limit'], x['battery_limit_kw']], axis=1).min(axis=1)
    x['hour_id']=x['hour'].astype(int)
    x = x.sort_values('hour', ascending=True)
    x['gen_prevhour']=x['gen'].shift(-1)
    x['two_hour_gen']=x['gen']+x['gen_prevhour']
    
    x['two_hour_battery_limit'] = x['fake_soc']*battery_kwh
    x['two_hour_gen']=pd.concat([x['two_hour_gen'], x['two_hour_battery_limit']], axis=1).min(axis=1)
           
    x['two_hour_load']=x['load_limit']+x['load_limit'].shift(-1)
    x['two_hour_baseline']=x['baseline_limit']+x['baseline_limit'].shift(-1)
    x = x[(x['hour_id']>16) & (x['hour_id']<21)]
    x.drop(columns=['fake_soc'])
    return x

replacement_gen = get_2hour_gen(replacement_result, 'replacement', storage_kw, storage_kwh)
incremental_gen = get_2hour_gen(incremental_result, 'incremental', storage_kw, storage_kwh)

In [17]:
replacement_gen.to_csv('~/Downloads/PearlX_BL.csv')

,date,hour,gross_load,solar,storage,genset,net load,soc,unit,load,...,baseline_limit,fake_soc,battery_limit_kw,gen,hour_id,gen_prevhour,two_hour_gen,two_hour_battery_limit,two_hour_load,two_hour_baseline
4312,2021-04-29 00:00:00+00:00,17.0,1.162396,0.000275,4.471889,0.0,0.000000,0.456315,Pearl X,1.162122,...,1.325348,1,2.281576,1.162122,17,0.742524,1.904646,9,1.904646,2.110243
2296,2021-02-04 00:00:00+00:00,17.0,0.742524,0.000000,9.057476,0.0,0.000000,0.924232,Pearl X,0.742524,...,0.784896,1,4.621161,0.742524,17,1.546610,2.289135,9,2.289135,3.210805
6760,2021-08-09 00:00:00+00:00,17.0,1.825975,0.279365,8.253390,0.0,0.000000,0.842183,Pearl X,1.546610,...,2.425909,1,4.210913,1.546610,17,2.086251,3.632861,9,3.685864,4.512160
7096,2021-08-23 00:00:00+00:00,17.0,2.440620,0.301366,7.660746,0.0,0.000000,0.781709,Pearl X,2.139254,...,2.086251,1,3.908544,2.086251,17,0.632858,2.719109,9,2.782603,2.719109
448,2020-11-19 00:00:00+00:00,17.0,0.765876,0.122526,9.156651,0.0,0.000000,0.934352,Pearl X,0.643349,...,0.632858,1,4.671761,0.632858,17,0.863978,1.496836,9,1.507327,1.789906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,2021-07-08 00:00:00+00:00,20.0,1.886779,0.000000,2.017840,0.0,0.000000,0.205902,Pearl X,1.886779,...,1.982258,1,1.029510,1.029510,20,2.130929,3.160439,9,4.017708,4.208586
7123,2021-08-24 00:00:00+00:00,20.0,2.130929,0.000000,9.800000,0.0,2.130929,1.000000,Pearl X,2.130929,...,2.226328,1,5.000000,2.130929,20,0.169490,2.300419,9,4.421747,4.563377
7411,2021-09-05 00:00:00+00:00,20.0,2.290818,0.000000,0.332201,0.0,0.000000,0.033898,Pearl X,2.290818,...,2.337049,1,0.169490,0.169490,20,1.726572,1.896062,9,4.017390,4.346747
5779,2021-06-29 00:00:00+00:00,20.0,1.726572,0.000000,9.800000,0.0,1.726572,1.000000,Pearl X,1.726572,...,2.009698,1,5.000000,1.726572,20,0.234284,1.960856,9,4.237107,4.311171


In [28]:
def get_best_dispatch(df, period):    
    if period == "Month Hour":
        best_dispatch= df.groupby([pd.Grouper(key='date', freq='M'),
                                            'hour_id']).agg({
            'two_hour_gen':['max','idxmax']
        })
    elif period == "Month":
                best_dispatch= df.groupby([pd.Grouper(key='date', freq='M')]).agg({
            'two_hour_gen':['max','idxmax']
        })
    best_dispatch.columns = [' '.join(col).strip() for col in best_dispatch.columns.values]
    best_dispatch.reset_index().columns
    final = best_dispatch.merge(right=df.rename(columns={'date':'event_datetime'}), left_on = 'two_hour_gen idxmax',
                                                   right_index=True,
                               validate="1:1")
    final = final.reset_index(drop=True)
    final['month']=final['event_datetime'].dt.strftime('%m')
    return final
rep_month_hour = get_best_dispatch(replacement_gen, "Month Hour")
inc_month_hour = get_best_dispatch(incremental_gen, "Month Hour")
rep_month = get_best_dispatch(replacement_gen, "Month")
inc_month = get_best_dispatch(incremental_gen, "Month")

rep_month_hour.to_csv('~/Downloads/PearlX_Sizing.csv')


,two_hour_gen max,two_hour_gen idxmax,event_datetime,hour,gross_load,solar,storage,genset,net load,soc,...,fake_soc,battery_limit_kw,gen,hour_id,gen_prevhour,two_hour_gen,two_hour_battery_limit,two_hour_load,two_hour_baseline,month
0,3.052792,40,2020-11-02 00:00:00+00:00,17.0,0.833594,0.200314,9.166720,0.0,0.000000,0.935380,...,1,4.676898,0.633280,17,2.419512,3.052792,9.8,3.090354,NaN,11
1,3.478540,65,2020-11-03 00:00:00+00:00,18.0,0.883907,0.000000,8.348119,0.0,0.000000,0.851849,...,1,4.259244,0.883907,18,2.594633,3.478540,9.8,3.854555,NaN,11
2,2.858156,546,2020-11-23 00:00:00+00:00,19.0,0.823622,0.000000,7.298015,0.0,0.000000,0.744695,...,1,3.723477,0.788986,19,2.069170,2.858156,9.8,2.892791,2.876843,11
3,2.612552,715,2020-11-30 00:00:00+00:00,20.0,0.820750,0.000000,6.432991,0.0,0.000000,0.656428,...,1,3.282138,0.760308,20,1.852244,2.612552,9.8,2.672995,2.701511,11
4,3.000145,1408,2020-12-29 00:00:00+00:00,17.0,0.906583,0.000000,8.893417,0.0,0.000000,0.907492,...,1,4.537458,0.900801,17,2.099344,3.000145,9.8,3.210135,3.000145,12
5,3.395617,1073,2020-12-15 00:00:00+00:00,18.0,1.079945,0.000000,7.479562,0.0,0.000000,0.763221,...,1,3.816103,1.021645,18,2.373972,3.395617,9.8,3.594056,3.395617,12
6,3.199593,1122,2020-12-17 00:00:00+00:00,19.0,1.208649,0.000000,6.867909,0.0,0.000000,0.700807,...,1,3.504035,0.995578,19,2.204015,3.199593,9.8,3.412664,3.412907,12
7,2.543986,1195,2020-12-20 00:00:00+00:00,20.0,0.865935,0.000000,6.451422,0.0,0.000000,0.658308,...,1,3.291542,0.757088,20,1.786898,2.543986,9.8,3.155904,2.799784,12
8,2.955129,1600,2021-01-06 00:00:00+00:00,17.0,0.848129,0.000000,8.951871,0.0,0.000000,0.913456,...,1,4.567281,0.848129,17,2.106999,2.955129,9.8,3.568325,3.055181,01
9,3.407906,1649,2021-01-08 00:00:00+00:00,18.0,0.915578,0.000000,8.094227,0.0,0.000000,0.825942,...,1,4.129708,0.915578,18,2.492329,3.407906,9.8,3.426576,3.481915,01


In [18]:
#Calculate Capacity revenues using 2021 Supply forecast summary
def get_capacity_values():
    dirname=os.getcwd()
    pge = pd.read_excel(dirname+"/2021 Supply Forecast V4.xlsx", skiprows=53,nrows=3)
    pge = pge.T.reset_index()[['index', 1]][5:17]
    pge['iou']='pge'
    sce = pd.read_excel(dirname+"/2021 Supply Forecast V4.xlsx", skiprows=70,nrows=3)
    sce = sce.T.reset_index()[['index', 1]][5:17]
    sce['iou']='sce'

    sdge = pd.read_excel(dirname+"/2021 Supply Forecast V4.xlsx", skiprows=87,nrows=3)
    sdge = sdge.T.reset_index()[['index', 1]][5:17]
    sdge['iou']='sdge'
    ious = pd.concat([pge, sce, sdge], axis=0)
    ious.columns = ['month', 'dollars_per_MW', 'iou']
    ious['month']=pd.to_datetime(ious['month']).dt.strftime('%m')
    return ious
ious = get_capacity_values()
inc_month

TypeError: invalid string coercion to datetime

In [15]:
def get_capacity_revenues(df, ious):
    df_capacity = df.merge(right = ious[ious['iou']=="pge"], on="month")
    df_capacity['revenue']=df_capacity['two_hour_gen']*df_capacity['dollars_per_MW']/1000
    return df_capacity
iou_cap = get_capacity_revenues(inc_month, ious)
rep_cap = get_capacity_revenues(rep_month, ious)

NameError: name 'ious' is not defined

In [101]:
#Calculate Energy revenues using active-bidder package

def get_energy_revenues(test, dispatches):
    dispatches = 12
    test = test[['month','two_hour_gen']]
    test['month']=test['month'].astype(int)
    test['dispatches']=dispatches
    test['location']='SLAP_PGCC'
    test['MW']=test['two_hour_gen']/1000
    df = pd.DataFrame()
    for i in range(0, test.shape[0]):
        lmp_data = pd.read_csv('/Users/jayraju/Documents/dev/GitHub/slap_lmps/'+"SLAP_PGCC"+'-APND-'+"DAM"+'.csv')
        location = test.iloc[i, 3]
        MW=test.iloc[i,4]
        dispatches=int(test.iloc[i,2])
        month= int(test.iloc[i, 0])

        lmp_data['PRC'] = lmp_data['MW']
        lmp_data.drop('MW', axis=1)

        lmp_data['start_datetime'] = pd.to_datetime(lmp_data['INTERVALSTARTTIME_GMT'],
                                                        format='%Y-%m-%dT%H:%M:%S') - pd.Timedelta(hours=7)
        lmp_data['month']=lmp_data['start_datetime'].dt.month

        lmp_data = lmp_data[lmp_data['month'] == month]

        bid_price = lmp_data.sort_values('PRC', ascending=False).iloc[(dispatches)-1,17]
        lmp_data['dispatch'] = lmp_data.PRC >= bid_price

        lmp_data['dispatch'] = lmp_data['dispatch'].astype(int)
        lmp_data['dispatch_rev']=0
        lmp_data.loc[
            lmp_data.dispatch == True,
            'dispatch_rev'
        ] = lmp_data.loc[
            lmp_data.dispatch == True,
            'PRC'
        ]
        dispatch_count = sum(lmp_data['dispatch'])
        revenue_mwh = round(sum(lmp_data['dispatch_rev']), 2)
        revenue=round(sum(lmp_data['dispatch_rev'], 2))*MW

        #if dispatches==2:
        #    print(month)
        #    print(lmp_data[lmp_data['dispatch_rev']>0]['OPR_HR'])
        #    print(lmp_data[lmp_data['dispatch_rev']>0]['dispatch_rev'])

        df = df.append(pd.DataFrame([[location, 
                                 unit, 
                                 MW,
                                 dispatches,
                                 month,
                                 bid_price, dispatch_count, revenue_mwh, revenue]],
                                   columns = ['location',
                                             'unit',
                                             'mw',
                                              'dispatch_case',
                                             'month',
                                             'bid_price',
                                             'dispatches',
                                             'revenue_mwh',
                                             'revenue']))
    return df 
inc_energy = 

In [19]:
#Plot things!

if 'MW' in lmp_data.columns:
    lmp_data['PRC'] = lmp_data['MW']
    lmp_data.drop('MW', axis=1)
lmp_data['start_datetime'] = pd.to_datetime(lmp_data['INTERVALSTARTTIME_GMT'],
                                                format='%Y-%m-%dT%H:%M:%S') - pd.Timedelta(hours=7)
lmp_data['month']=lmp_data['start_datetime'].dt.month
bid_price = lmp_data.sort_values('PRC', ascending=False).iloc[(dispatches)-1,17]
lmp_data['dispatch'] = lmp_data.PRC >= bid_price
lmp_data['dispatch'] = lmp_data['dispatch'].astype(int)
lmp_data['dispatch_rev']=0
lmp_data.loc[
    lmp_data.dispatch == True,
    'dispatch_rev'
] = lmp_data.loc[
    lmp_data.dispatch == True,
    'PRC'
]
dispatch_count = sum(lmp_data['dispatch'])
revenue_mwh = round(sum(lmp_data['dispatch_rev']), 2)
revenue=round(sum(lmp_data['dispatch_rev'], 2))*MW

NameError: name 'lmp_data' is not defined

In [ ]:
#Export Final CSV

filenames = os.listdir(path)